### Section 1: Preparation

In [1]:
##########################################################################################################
#
#         2023-5 v1: Base version
#         2023-5 v2: Calculates all years data of revenue, eps and freeCashFlow based on growth number 
#
##########################################################################################################

import pandas as pd
import numpy as np
from datetime import date

import FundamentalAnalysis as fa

# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
# Input section. Enter ticker, start_date (normally first IPO date) and end_date (normally last year)
ticker = 'BRK-B'
api_key = "99468a6d827666a6ee01f6788171dc40"

start_date = '1996-01-01'
end_date = '2022-12-31'

In [3]:
# Write single dataframe to excel sheet
def write_a_dataframe_to_excel(df, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter', 
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        worksheet.write_string(row, COLUMN, df.name)
        row += 1
        df.to_excel(writer, sheet_name=sheetname,
                    startrow=row, startcol=COLUMN)
        
# Automatically write multiple dataframe to excel sheet
def write_dataframes_to_excel(dataframes, filename, sheetname):
    with pd.ExcelWriter(f'{filename}.xlsx', 
                        engine='xlsxwriter',
                        datetime_format='yyyy') as writer:
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheetname)
        writer.sheets[sheetname] = worksheet

        COLUMN = 0
        row = 0

        for df in dataframes:
            worksheet.write_string(row, COLUMN, df.name)
            row += 1
            df.to_excel(writer, sheet_name=sheetname,
                        startrow=row, startcol=COLUMN,
                       float_format = '%.2f')
            row += df.shape[0] + 2

# Apply percentage format to number
def percentage_format(x):
    if pd.isna(x):
        return x
    else:
        return '{:.2%}'.format(x)

# Update NaN value of revenue, eps and freeCashFlow based on growth calculation
# Reason: FinancialModellingPrep's API only provide 5 years data for free account. But provides all years data for growth.
# This function calculates all years data of revenue, eps and freeCashFlow based on growth
def update_per_growth(df, rownum):
    for col in range(1, len(df.columns)):
        if pd.isna(df.iloc[rownum][col]):
            df.iloc[rownum, col] = df.iloc[rownum][col-1] / (df.iloc[rownum+1][col-1]+1)
            
    return df

In [4]:
import yfinance as yf

# stock = yf.Ticker('AAPL')
stock = yf.download(ticker, start_date, end_date)      # get stock history data up to last year
stock_today = yf.download(ticker, end_date)            # get stock data for current year

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Section 2: Get price info

In [5]:
# Obtain last day in Dec. of each year
stock['year'] = stock.index.year
stock['month'] = stock.index.month
stock['day'] = stock.index.day
max_day_of_year = stock.loc[stock.index.month == 12].groupby('year').max()['day']

# Get historical stock price per year (year end price)
stock_close = stock.loc[(stock.index.month == 12) & (stock.index.day == max_day_of_year[stock.index.year])]
# stock_close = pd.concat([stock_close,stock_today])
stock_close = pd.concat([stock_close, stock_today.loc[stock_today.iloc[-1].name.strftime("%Y-%m-%d"):]])
stock_price = pd.DataFrame()
stock_price['Price'] = stock_close['Close']

# Formatting
stock_price = stock_price.sort_index(ascending=False).transpose()
# stock_price = pd.concat([stock_price.iloc[:-3].sort_index(ascending=False), stock_price.iloc[-3:]]).transpose()

# Get price percentage changes per year
returns = pd.DataFrame()
returns['PriceGrowth'] = stock_close['Close'].pct_change()

# Formatting
returns['PriceGrowth'] = returns['PriceGrowth'].apply(lambda x: percentage_format(x))
# returns = pd.concat([returns.iloc[:-3].sort_index(ascending=False), returns.iloc[-3:]]).transpose()
returns = returns.sort_index(ascending=False).transpose()

# Combine 2 rows
price_info = pd.concat([stock_price, returns])

# Change column names from timestamp format to 4 digit year
price_info.rename(columns=lambda s: str(s)[:4], inplace=True)
price_info

Date,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996
Price,322.64,308.9,299,231.87,226.5,204.18,198.22,162.98,132.04,150.15,...,58.71,58.72,56.3,48.46,50.5,47.08,36.6,47,30.78,22.24
PriceGrowth,4.45%,3.31%,28.95%,2.37%,10.93%,3.01%,21.62%,23.43%,-12.06%,26.64%,...,-0.02%,4.30%,16.18%,-4.04%,7.26%,28.63%,-22.13%,52.70%,38.40%,NaN


### Section 3: Bring all finanical data needed for instrinsic value calculation

In [6]:
# Loan other finanical data
income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
entreprise_value = fa.enterprise(ticker, api_key)
balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")

# Store financial data in results
results = pd.DataFrame()
results = pd.concat([income_statement_annually.loc['revenue'],
                     growth_annually.loc['revenueGrowth'], 
                     income_statement_annually.loc['netIncome'],
                     income_statement_annually.loc['eps'],
                     growth_annually.loc['epsgrowth'],
                     key_metrics_annually.loc['bookValuePerShare'],
                     growth_annually.loc['bookValueperShareGrowth'],
                     cash_flow_statement_annually.loc['freeCashFlow'],
                     growth_annually.loc['freeCashFlowGrowth'],
                     key_metrics_annually.loc['freeCashFlowPerShare'],                     
                     key_metrics_annually.loc['marketCap'],
                     key_metrics_annually.loc['enterpriseValue'],
                     key_metrics_annually.loc['peRatio'],
                     key_metrics_annually.loc['pbRatio'],
                     key_metrics_annually.loc['roe'],
                     balance_sheet_annually.loc['cashAndShortTermInvestments'],
                     balance_sheet_annually.loc['totalDebt'],
                     entreprise_value.loc['numberOfShares']
#                      dcf_annually.loc['DCF']
                    ], axis=1)

# Function to calculate eeRatio = enterpriseValue/netIncome
def calc_eeRatio(df):
    df['eeRatio'] = df['enterpriseValue'] / df['netIncome']
    return df

# Update eeRatio
results.rename({'pbRatio': 'eeRatio'}, axis='columns', inplace=True)
results = calc_eeRatio(results).transpose()

# results = results.transpose()
results = pd.concat([price_info, results])

# Calculate all years data for revenue, eps and freeCashFlow
update_per_growth(results, 2)             # revenue
update_per_growth(results, 5)             # eps
update_per_growth(results, 9)             # freeCashFlow

# Format decimal fields format as percentage
results.loc['revenueGrowth'] = results.loc['revenueGrowth'].apply(lambda x: percentage_format(x))
results.loc['epsgrowth'] = results.loc['epsgrowth'].apply(lambda x: percentage_format(x))
results.loc['bookValueperShareGrowth'] = results.loc['bookValueperShareGrowth'].apply(lambda x: percentage_format(x))
results.loc['freeCashFlowGrowth'] = results.loc['freeCashFlowGrowth'].apply(lambda x: percentage_format(x))
results.loc['roe'] = results.loc['roe'].apply(lambda x: percentage_format(x))

In [7]:
results

,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1997,1996,1995,1994,1993,1992,1991,1990,1989,1988
Price,322.64,308.9,299,231.87,226.5,204.18,198.22,162.98,132.04,150.15,...,30.78,22.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PriceGrowth,4.45%,3.31%,28.95%,2.37%,10.93%,3.01%,21.62%,23.43%,-12.06%,26.64%,...,38.40%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
revenue,NaN,234190000000,354636000000,286256000000,327223000000,225382000000,2.42137e+11,2.23604e+11,2.10821e+11,1.94673e+11,...,1.07376e+10,1.07974e+10,3.7134e+09,3.8475e+09,2.6192e+09,3.0293e+09,2.4275e+09,1.5801e+09,2.4839e+09,2.4649e+09
revenueGrowth,NaN,-33.96%,23.89%,-12.52%,45.19%,-6.92%,8.29%,6.06%,8.29%,6.88%,...,-0.55%,190.77%,-3.49%,46.90%,-13.54%,24.79%,53.63%,-36.39%,0.77%,0.00%
netIncome,NaN,-22058000000,89795000000,42521000000,81417000000,4021000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
eps,NaN,-10.01,39.64,17.78,33.22,1.63067,18.2173,9.76334,9.77067,8.06134,...,1.028,1.37667,0.446407,0.313027,0.448,0.236827,0.256,0.229333,0.260007,0.232153
epsgrowth,NaN,-125.25%,122.95%,-46.48%,1937.20%,-91.05%,86.59%,-0.08%,21.20%,2.04%,...,-25.33%,208.39%,42.61%,-30.13%,89.17%,-7.49%,11.63%,-11.80%,12.00%,0.00%
bookValuePerShare,NaN,214.386,223.461,185.292,173.319,141.422,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bookValueperShareGrowth,NaN,-4.06%,20.60%,6.91%,22.55%,0.17%,23.01%,10.70%,6.42%,8.25%,...,31.26%,22.14%,40.88%,27.20%,14.42%,20.29%,39.34%,0.00%,0.00%,0.00%
freeCashFlow,NaN,21760000000,26145000000,26761000000,22708000000,22863000000,3.4068e+10,1.9581e+10,1.5409e+10,1.6825e+10,...,2.3356e+09,1.26e+09,1.1279e+09,9.291e+08,7.262e+08,8.773e+08,5.482e+08,5.226e+08,5.226e+08,5.226e+08


### Section 4: Compound Annual Growth Rate (CAGR) for Price, Revenue, EPS & Free Cash Flow

In [8]:
# Calc Compound Annual Growth Rate (CAGR)
# def CAGR(first, last, periods):
#     return (last/first)**(1/periods)-1
def CAGR(first, last, periods):
    result = 0
    
    if first > 0 and last > 0:
        result = (last/first)**(1/periods)-1
    elif first < 0 and last < 0:
        result = -1 * ((abs(last)/abs(first))**(1/periods)-1)
    elif first < 0 and last > 0:
        result = ((last+2*abs(first))/abs(first))**(1/periods)-1
    elif first > 0 and last < 0:
        result = -1 * (((abs(last)+2*first)/first)**(1/periods)-1)
        
    return result


# Get last not null column
def earliest_col_notna(df,idx):
    for col in reversed(df.columns):
        if pd.notna(df.loc[idx][col]):
            return df.columns.get_loc(col)
        
# Calc CAGR (5/10/All years) for growth related columns
def calc_CAGR_for_col(df, idx):
    earliest_year = earliest_col_notna(df, idx)
    
    five_yrs_CAGR = np.nan
    ten_yrs_CAGR = np.nan
    All_CAGR = np.nan
    
    if earliest_year < 6:
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    elif earliest_year < 11:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
    else:
        five_yrs_CAGR = CAGR(df.loc[idx][6], df.loc[idx][1], 5)
        ten_yrs_CAGR = CAGR(df.loc[idx][11], df.loc[idx][1], 10)
        All_CAGR = CAGR(df.loc[idx][earliest_year], df.loc[idx][1], earliest_year-1)
        
    return [five_yrs_CAGR, ten_yrs_CAGR, All_CAGR]

In [9]:
# Calc average returns from growth related columns
returns_lists = []

returns_lists.append(calc_CAGR_for_col(results,'Price'))
returns_lists.append(calc_CAGR_for_col(results,'revenue'))
returns_lists.append(calc_CAGR_for_col(results,'eps'))
returns_lists.append(calc_CAGR_for_col(results,'freeCashFlow'))

avg_returns = pd.DataFrame(returns_lists, 
             index=['PriceGrowth','revenueGrowth','epsgrowth','freeCashFlowGrowth'], 
             columns=['5 years Avg','10 years Avg','All years Avg'])

# Formatting
avg_returns['5 years Avg'] = avg_returns['5 years Avg'].apply(lambda x: percentage_format(x))
avg_returns['10 years Avg'] = avg_returns['10 years Avg'].apply(lambda x: percentage_format(x))
avg_returns['All years Avg'] = avg_returns['All years Avg'].apply(lambda x: percentage_format(x))

### Last section: Put all data together and save to excel

In [10]:
# Set report section titles
results.name = ticker + ' Financial data'
avg_returns.name = ticker + ' Compound Annual Growth Rate (CAGR)'

dataframes = (results, avg_returns)
write_dataframes_to_excel(dataframes, 
                          'data\Stock fundamental analysis - '+ ticker + ' ' + date.today().strftime("%Y-%m"), 
                          ticker)